In [3]:
from utils.branch_bound import matrix_table, branch_bound_tree

In [1]:
from collections import defaultdict

In [2]:
import numpy as np

In [3]:
s = '11101010101'
array_of_integers = np.array(list(s), dtype=int)
print(array_of_integers)

[1 1 1 0 1 0 1 0 1 0 1]


In [4]:
integers = [0, 1, 2, 3, 6, 7, 8, 9, 14, 15]
max_int = max(integers)
print(max_int)
max_bit = format(max_int, 'b')
print(max_bit)
print(len(max_bit))
bit_format = str(len(max_bit))+'b'
print(bit_format)
# Convert integers to binary representations
binary_representations = {integer: format(integer, bit_format).replace(' ', '0') for integer in integers}
print(binary_representations)


15
1111
4
4b
{0: '0000', 1: '0001', 2: '0010', 3: '0011', 6: '0110', 7: '0111', 8: '1000', 9: '1001', 14: '1110', 15: '1111'}


In [5]:
from collections import defaultdict

In [19]:
def build_binary_table(prime_implicants):
    max_int = max(prime_implicants)
    max_bit = format(max_int, 'b')
    bit_format = str(len(max_bit))+'b'
    binary_representations = {integer: format(integer, bit_format).replace(' ', '0') for integer in prime_implicants}
    return binary_representations

In [20]:
integers = [0, 1, 2, 3, 6, 7, 8, 9, 14, 15]
binary_table = build_binary_table(integers)
print(binary_table)

{0: '0000', 1: '0001', 2: '0010', 3: '0011', 6: '0110', 7: '0111', 8: '1000', 9: '1001', 14: '1110', 15: '1111'}


In [21]:
def build_initial_pi(binary_lists):
    pi_table = defaultdict(lambda: defaultdict(str))
    for minterm, bits in binary_lists.items():
        uninverted_bits_count = sum([int(char) for char in bits])
        pi_table[uninverted_bits_count][(minterm,)]=bits
    return pi_table

In [22]:
pi_table = build_initial_pi(binary_table)
print(pi_table)

defaultdict(<function build_initial_pi.<locals>.<lambda> at 0x0000023EFA464940>, {0: defaultdict(<class 'str'>, {(0,): '0000'}), 1: defaultdict(<class 'str'>, {(1,): '0001', (2,): '0010', (8,): '1000'}), 2: defaultdict(<class 'str'>, {(3,): '0011', (6,): '0110', (9,): '1001'}), 3: defaultdict(<class 'str'>, {(7,): '0111', (14,): '1110'}), 4: defaultdict(<class 'str'>, {(15,): '1111'})})


In [28]:
def simplify_prime_implicants(pi_table):
    uninv_bits_counts = list(pi_table.keys())
    next_pi_table = defaultdict(lambda: defaultdict(str))
    unique_bits = set()
    leftover_check = set(minterms for group_minterms in pi_table.values() for minterms in group_minterms.keys())
    print(leftover_check)
    for idx in range(len(uninv_bits_counts)-1):
        current_group, next_group = uninv_bits_counts[idx], uninv_bits_counts[idx+1]
        for current_minterms, current_bits in pi_table[current_group].items():
            for next_minterms, next_bits in pi_table[next_group].items():
                count = 0
                variable = ''
                for byte_idx in range(0,4):
                    current_byte = current_bits[byte_idx]
                    next_byte = next_bits[byte_idx]
                    if current_byte == next_byte:
                        variable += current_bits[byte_idx]
                    else:
                        variable += '-'
                        count += 1

                if count == 1:
                    if variable not in unique_bits:
                        # next_group_table[idx].append((current_minterms+next_minterms))
                        # next_minterm_table[current_minterms+next_minterms] = variable
                        next_pi_table[idx][current_minterms+next_minterms] = variable
                        unique_bits.add(variable)

                    leftover_check.discard(current_minterms), leftover_check.discard(next_minterms)
                    
    return next_pi_table, leftover_check

In [54]:
a = set((1,3,4,5))
b = set((2,3,4,5), (23,3,45,))

TypeError: set expected at most 1 argument, got 2

In [43]:
next_pi_table, last_group_check = simplify_prime_implicants(pi_table)
print(next_pi_table)
print(last_group_check)

{(6,), (2,), (15,), (8,), (14,), (1,), (7,), (0,), (3,), (9,)}
defaultdict(<function simplify_prime_implicants.<locals>.<lambda> at 0x0000023EFC596B80>, {0: defaultdict(<class 'str'>, {(0, 1): '000-', (0, 2): '00-0', (0, 8): '-000'}), 1: defaultdict(<class 'str'>, {(1, 3): '00-1', (1, 9): '-001', (2, 3): '001-', (2, 6): '0-10', (8, 9): '100-'}), 2: defaultdict(<class 'str'>, {(3, 7): '0-11', (6, 7): '011-', (6, 14): '-110'}), 3: defaultdict(<class 'str'>, {(7, 15): '-111', (14, 15): '111-'})})
set()


In [44]:
next_pi_table2, last_group_check2 = simplify_prime_implicants(next_pi_table)

{(0, 1), (3, 7), (6, 14), (14, 15), (2, 3), (6, 7), (0, 2), (8, 9), (2, 6), (7, 15), (0, 8), (1, 3), (1, 9)}


In [45]:
next_pi_table2

defaultdict(<function __main__.simplify_prime_implicants.<locals>.<lambda>()>,
            {0: defaultdict(str, {(0, 1, 2, 3): '00--', (0, 1, 8, 9): '-00-'}),
             1: defaultdict(str, {(2, 3, 6, 7): '0-1-'}),
             2: defaultdict(str, {(6, 7, 14, 15): '-11-'})})

In [49]:
test = {minterm:bits for minterms in next_pi_table2.values() for minterm, bits in minterms.items()}
print(test)

{(0, 1, 2, 3): '00--', (0, 1, 8, 9): '-00-', (2, 3, 6, 7): '0-1-', (6, 7, 14, 15): '-11-'}


In [53]:
for idx, minterm in enumerate(test.items()):
    print(idx)
    print(minterm)

0
((0, 1, 2, 3), '00--')
1
((0, 1, 8, 9), '-00-')
2
((2, 3, 6, 7), '0-1-')
3
((6, 7, 14, 15), '-11-')


In [39]:
test = set(minterm for minterms in next_pi_table2.values() for minterm in minterms.keys() )

In [57]:
test = [((2, 3, 6, 7,0, 1, 8, 9), '0-1-'), ((0, 1, 2, 3), '00--'), ((0, 1, 8, 9), '-00-'),((2, 3, 6, 7), '0-1-')]
minterm_table_sorted = sorted(test, key=lambda x: (len(x[0]), x[0]))
print(minterm_table_sorted)

[((0, 1, 2, 3), '00--'), ((0, 1, 8, 9), '-00-'), ((2, 3, 6, 7), '0-1-'), ((2, 3, 6, 7, 0, 1, 8, 9), '0-1-')]


In [33]:
last_group_check2

set()

In [35]:
max_bit = format(max_int, 'b')
most_significant_bit = len(max_bit)
minterm_matrix = np.zeros((most_significant_bit,len(integers)), dtype='int')
print(minterm_matrix)

[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]


In [36]:
integers.index(7)

5

In [37]:
for idx, minterm in enumerate(next_minterm_table2.keys()):
    row = minterm_matrix[idx, :]
    for bit in minterm:
        row_idx = integers.index(bit)
        row[row_idx] = 1
    

In [38]:
integers

[0, 1, 2, 3, 6, 7, 8, 9, 14, 15]

In [39]:
next_minterm_table2.keys()

dict_keys([(0, 1, 2, 3), (0, 1, 8, 9), (2, 3, 6, 7), (6, 7, 14, 15)])

In [66]:
minterm_matrix

array([[1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 0, 0, 0, 1, 1, 0, 0],
       [0, 0, 1, 1, 1, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 1, 0, 0, 1, 1]])

In [67]:
col_sums = np.sum(minterm_matrix, axis = 0)
min_col_sums = min(col_sums)
print(min_col_sums)
indicies = np.where(col_sums == min_col_sums)[0]
print(indicies)

1
[6 7 8 9]


In [68]:
next_matrix = np.delete(minterm_matrix, indicies, axis = 1)
print(next_matrix)

[[1 1 1 1 0 0]
 [1 1 0 0 0 0]
 [0 0 1 1 1 1]
 [0 0 0 0 1 1]]


In [69]:
integers

[0, 1, 2, 3, 6, 7, 8, 9, 14, 15]

In [73]:
next_prime_implicants = np.delete(integers, indicies, axis = 0)
print(next_prime_implicants)

[0 1 2 3 6 7]


10

In [50]:
test = minterm_matrix[: , -1]
print(test)
idx = np.where(test == 1)[0][0]
print(idx)

[0 0 0 1]
3


In [31]:
next_group_table3, next_minterm_table3 = simplify_prime_implicants(next_group_table2, next_minterm_table2)

In [32]:
next_group_table3

defaultdict(list, {})

In [33]:
next_minterm_table3

{}

In [143]:
unique_check = set()
for minterms in next_minterm_table2.keys():
    for group in minterms:
        if group not in unique_check:
            unique_check.add(group)
        else:
            unique_check.remove(group)
print(unique_check)

{8, 9, 14, 15}


In [147]:
keys_unique_groups = []
vals_unique_groups = []
for key in next_minterm_table2:
    if any(group in unique_check for group in key):
        keys_unique_groups.append(key)
        vals_unique_groups.append(next_minterm_table2[key])
print(keys_unique_groups)
print(vals_unique_groups)

[(0, 1, 8, 9), (6, 7, 14, 15)]
['-00-', '-11-']


In [150]:
results = []
for minterms, bits in next_minterm_table2.items():
    final_result = ''
    if minterms not in keys_unique_groups:
        for unique_key in keys_unique_groups:
            final_result += next_minterm_table2[unique_key] + '+'
        final_result += bits
        results.append(final_result)

In [151]:
print(results)

['-00-+-11-+00--', '-00-+-11-+0-1-']
